# Exploring conformational space of selected macrocycles - "M13"; <br /> Part 1: Generation and selection of conformer candidates (MM methods)

In [135]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import glob
import py3Dmol

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Mon Jun 19 10:38:42 2017


In [2]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def write_to_dict(prefix, suppl):
    moldict = {}
    for i, mol in enumerate(suppl):
        name = prefix + str(i)
        moldict[name] = mol
    return moldict


def align_structures_to_crystal(moldict):
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))


def align_structures_to_lowest_energy(moldict, energy_dict):
    """
    align all structures in "moldict" to the one of the lowest energy
    """
    energy_sorted = sorted(energy_dict.items(), key=lambda x: x[1])
    first = energy_sorted[0][0]
    core_first = moldict[first].GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,moldict[first],atomMap=list(zip(core_mol,core_first)))


def prepare_view(moldict):
    p = py3Dmol.view(width=400,height=400)
    for key, mol in moldict.items():
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.15'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p        


def make_similarity_matrix(moldict):
    
    similarity_matrix = {}

    for k1, m13 in moldict.items():
        for k2, m2 in moldict.items():
            if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
                pass
            else:
                if k1 != k2:
                    rms = AllChem.GetBestRMS(Chem.RemoveHs(m13),Chem.RemoveHs(m2))
                    similarity_matrix[(k1, k2)] = rms 
                    
    return similarity_matrix


def find_duplicates_in_sorted_similarity_matrix(similarity_matrix_sorted, energy):
    
    similarity_thresh = 0.5 # Angstrom
    energy_thresh     = 5   # kcal/mol
    
    to_be_deleted     = []
    
    for pair in similarity_matrix_sorted:
        if pair[1] < similarity_thresh:
            conf1 = pair[0][0]
            conf2 = pair[0][1]
            if abs(energy[conf1] - energy[conf2]) < energy_thresh:
                #print("conf1, conf2, E(conf1), E(conf2) = ", conf1, conf2, energy_m13_b[conf1], energy_m13_b[conf2])
                if energy[conf1] < energy[conf2]:
                    to_be_deleted.append(conf2)
                else:
                    to_be_deleted.append(conf1)
        else:
            # similarity_matrix_b_sorted is sorted, so here we would already start looping over 
            # pairs for which rmsd is > threshold 
            # and we do not need to do this, so break
            break

    return to_be_deleted


def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

## Crystal structure of "m13" macrocycle

In [3]:
cm13 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.xyz','r').read()
vcm13 = py3Dmol.view(width=400,height=400)
vcm13.removeAllModels()
vcm13.addModel(cm13,'xyz')
vcm13.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm13.setBackgroundColor('0xeeeeee')
vcm13.zoomTo()
vcm13.show()

In [4]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'N1C(=O)c2cc(C(=O)NCCCCCNC(=O)c3cc(C(=O)NCCCCC1)cc(c3)C(C)(C)C)cc(c2)C(C)(C)C'
core_smiles = '[c]1[c][c][c]c([c]1)C([C])([C])[C]'

m13 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m13 = m13.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m13 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.sdf')
m13_crystal = templ_m13[0]

## Conformers generated with the Balloon software:

Conformers were generated in the Balloon software with the following starting structures:

* the crystal geometry (3D), results with prefix: "m13_b_sdf"; the crystal is of the "ss-ss" type;

* the SMILES signature of m13 ("0D"), program was allowed to "rebuild the geometry" (option "--rebuildGeometry"), results with prefix: "m13_b_smi"

* 3D structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

  where "ss\_sa" means "(ring1: syn-syn)\_(ring2: syn-anti)" configuration of the amide groups, paired by the neighbouring aromatic ring, etc.
    
In all cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

The generated conformer structures are presented in a separate notebook: [link](http://nbviewer.jupyter.org/github/gosiao/icho-notebooks/blob/master/conformers_m13_suppl1.ipynb).

In [5]:
inps_m13_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsdf/*.sdf')
inps_m13_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsmiles/*.sdf')
inps_m13_b_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_ss_sa/*.sdf')
inps_m13_b_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_ss_aa/*.sdf')
inps_m13_b_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_sa_sa/*.sdf')
inps_m13_b_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_sa_as/*.sdf')
inps_m13_b_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_sa_aa/*.sdf')
inps_m13_b_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_m13_aa_aa/*.sdf')

In [6]:
e_m13_b_sdf = grep_energies_from_sdf_outputs(inps_m13_b_sdf)
e_m13_b_smi = grep_energies_from_sdf_outputs(inps_m13_b_smi)
e_m13_b_ss_sa = grep_energies_from_sdf_outputs(inps_m13_b_ss_sa)
e_m13_b_ss_aa = grep_energies_from_sdf_outputs(inps_m13_b_ss_aa)
e_m13_b_sa_sa = grep_energies_from_sdf_outputs(inps_m13_b_sa_sa)
e_m13_b_sa_as = grep_energies_from_sdf_outputs(inps_m13_b_sa_as)
e_m13_b_sa_aa = grep_energies_from_sdf_outputs(inps_m13_b_sa_aa)
e_m13_b_aa_aa = grep_energies_from_sdf_outputs(inps_m13_b_aa_aa)

In [7]:
# write conformers to dictionaries
   
suppl_m13_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_sdfout.sdf')
suppl_m13_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_smilesout.sdf')
suppl_m13_b_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_ss_sa_sdfout.sdf')
suppl_m13_b_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_ss_aa_sdfout.sdf')
suppl_m13_b_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_sa_sa_sdfout.sdf')
suppl_m13_b_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_sa_as_sdfout.sdf')
suppl_m13_b_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_sa_aa_sdfout.sdf')
suppl_m13_b_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_aa_aa_sdfout.sdf')

allmol_m13_b_sdf   = write_to_dict("m13_b_sdf_", suppl_m13_b_sdf)
allmol_m13_b_smi   = write_to_dict("m13_b_smi_", suppl_m13_b_smi)
allmol_m13_b_ss_sa = write_to_dict("m13_b_ss_sa_", suppl_m13_b_ss_sa)
allmol_m13_b_ss_aa = write_to_dict("m13_b_ss_aa_", suppl_m13_b_ss_aa)
allmol_m13_b_sa_sa = write_to_dict("m13_b_sa_sa_", suppl_m13_b_sa_sa)
allmol_m13_b_sa_as = write_to_dict("m13_b_sa_as_", suppl_m13_b_sa_as)
allmol_m13_b_sa_aa = write_to_dict("m13_b_sa_aa_", suppl_m13_b_sa_aa)
allmol_m13_b_aa_aa = write_to_dict("m13_b_aa_aa_", suppl_m13_b_aa_aa)

### pre-screening

All generated structures are pre-optimized with MM methods (MMFF94-like force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account non-hydrogen atoms only);
* compare the energies of conformers from pairs which are found to be similar (RMSD lower than the threshold);
* if these energies are too similar (the difference lower than the threshold), we will remove the conformer which has the higher energy value;

In [8]:
allmol_m13_b = {}
allmol_m13_b.update(allmol_m13_b_sdf)
allmol_m13_b.update(allmol_m13_b_smi)
allmol_m13_b.update(allmol_m13_b_ss_sa)
allmol_m13_b.update(allmol_m13_b_ss_aa)
allmol_m13_b.update(allmol_m13_b_sa_sa)
allmol_m13_b.update(allmol_m13_b_sa_as)
allmol_m13_b.update(allmol_m13_b_sa_aa)
allmol_m13_b.update(allmol_m13_b_aa_aa)

print("The total number of generated conformers (before the duplicates removal) = ", len(allmol_m13_b))
             
energy_m13_b = {}
energy_m13_b.update(e_m13_b_sdf)
energy_m13_b.update(e_m13_b_smi)
energy_m13_b.update(e_m13_b_ss_sa)
energy_m13_b.update(e_m13_b_ss_aa)
energy_m13_b.update(e_m13_b_sa_sa)
energy_m13_b.update(e_m13_b_sa_as)
energy_m13_b.update(e_m13_b_sa_aa)
energy_m13_b.update(e_m13_b_aa_aa)

The total number of generated conformers (before the duplicates removal) =  772


In [ ]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_b = make_similarity_matrix(allmol_m13_b)
similarity_matrix_b_sorted = sorted(similarity_matrix_b.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy
to_be_deleted = find_duplicates_in_sorted_similarity_matrix(similarity_matrix_b_sorted, energy_m13_b)

for mol in to_be_deleted:
    #print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_b.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_b[k]
    allmol_m13_b.pop(mol, None)
    energy_m13_b.pop(mol, None)
    

In [145]:
print("We have removed potential conformer duplicates.")
print("The final number of remaining conformers = ", len(allmol_m13_b))
print("Below we present all the remaining conformers aligned (to one aromatic ring).")

We have removed potential conformer duplicates.
The final number of remaining conformers =  54
Below we present all the remaining conformers aligned (to one aromatic ring).


In [146]:
align_structures_to_lowest_energy(allmol_m13_b, energy_m13_b)
p_b = prepare_view(allmol_m13_b)
p_b.show()

In [147]:
print("Sorted energy of all selected conformers and the energy differences with respect to the lowest:")
energy_b_sorted = sorted(energy_m13_b.items(), key=lambda x: x[1])
energy_b_diff = []
e_b_min = energy_b_sorted[0][1]
for e in energy_b_sorted:
    e_diff = e[1] - e_b_min
    energy_b_diff.append([e[0], e[1], e_diff])

e_b_df = pd.DataFrame(energy_b_diff, columns=["conformer", "E", "E - Emin"])
e_b_df

Sorted energy of all selected conformers and the energy differences with respect to the lowest:


,conformer,E,E - Emin
0,m13_b_sdf_0,53.045942,0.000000
1,m13_b_smi_0,53.057266,0.011324
2,m13_b_sdf_3,53.710066,0.664124
3,m13_b_sdf_6,56.225813,3.179871
4,m13_b_sa_as_1,56.380826,3.334885
5,m13_b_ss_sa_4,57.105316,4.059374
6,m13_b_ss_sa_5,57.150992,4.105050
7,m13_b_sa_sa_0,57.179163,4.133221
8,m13_b_sa_aa_0,58.268791,5.222849
9,m13_b_ss_sa_6,58.357893,5.311951


## Conformers generated with the RDKit software:

Conformers were generated in the RDKit software with the following starting structures:

* the crystal geometry (3D), results with prefix: "m13_b_sdf"; the crystal is of the "ss-ss" type;

* the SMILES signature of m13 ("0D"), program was allowed to "rebuild the geometry" (option "--rebuildGeometry"), results with prefix: "m13_b_smi"

* 3D structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

  where "ss\_sa" means "(ring1: syn-syn)\_(ring2: syn-anti)" configuration of the amide groups, paired by the neighbouring aromatic ring, etc.

In all cases the RDKit software was asked to generate 100 conformers using the distnce geometry algorithm with default settings (only "pruneRmsThresh" set to 1.0 in "AllChem.EmbedMultipleConfs" and "maxIters" set to 500 or 700 (when needed) in "AllChem.UFFOptimizeMolecule").

The generated conformer structures are presented in a separate notebook: [link](http://nbviewer.jupyter.org/github/gosiao/icho-notebooks/blob/master/conformers_m13_suppl1.ipynb).

In [148]:
inps_m13_rdkit_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_crystal_from_smiles/*.sdf')
inps_m13_rdkit_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_crystal_from_sdf/*.sdf')
inps_m13_rdkit_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_ss_sa/*.sdf')
inps_m13_rdkit_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_ss_aa/*.sdf')
inps_m13_rdkit_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_sa_sa/*.sdf')
inps_m13_rdkit_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_sa_as/*.sdf')
inps_m13_rdkit_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_sa_aa/*.sdf')
inps_m13_rdkit_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_from_m13_aa_aa/*.sdf')

In [149]:
e_m13_rdkit_smi = grep_energies_from_sdf_outputs(inps_m13_rdkit_smi)
e_m13_rdkit_sdf = grep_energies_from_sdf_outputs(inps_m13_rdkit_sdf)
e_m13_rdkit_ss_sa = grep_energies_from_sdf_outputs(inps_m13_rdkit_ss_sa)
e_m13_rdkit_ss_aa = grep_energies_from_sdf_outputs(inps_m13_rdkit_ss_aa)
e_m13_rdkit_sa_sa = grep_energies_from_sdf_outputs(inps_m13_rdkit_sa_sa)
e_m13_rdkit_sa_as = grep_energies_from_sdf_outputs(inps_m13_rdkit_sa_as)
e_m13_rdkit_sa_aa = grep_energies_from_sdf_outputs(inps_m13_rdkit_sa_aa)
e_m13_rdkit_aa_aa = grep_energies_from_sdf_outputs(inps_m13_rdkit_aa_aa)

In [150]:
# write conformers to dictionaries
suppl_m13_rdkit_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_smiles.sdf')
suppl_m13_rdkit_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_sdf.sdf')
suppl_m13_rdkit_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_ss_sa.sdf')
suppl_m13_rdkit_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_ss_aa.sdf')
suppl_m13_rdkit_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_sa_sa.sdf')
suppl_m13_rdkit_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_sa_as.sdf')
suppl_m13_rdkit_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_sa_aa.sdf')
suppl_m13_rdkit_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_m13_aa_aa.sdf')

allmol_m13_rdkit_smi = write_to_dict("m13_rdkit_smi_", suppl_m13_rdkit_smi)
allmol_m13_rdkit_sdf = write_to_dict("m13_rdkit_sdf_", suppl_m13_rdkit_sdf)
allmol_m13_rdkit_ss_sa = write_to_dict("m13_rdkit_ss_sa_", suppl_m13_rdkit_ss_sa)
allmol_m13_rdkit_ss_aa = write_to_dict("m13_rdkit_ss_aa_", suppl_m13_rdkit_ss_aa)
allmol_m13_rdkit_sa_sa = write_to_dict("m13_rdkit_sa_sa_", suppl_m13_rdkit_sa_sa)
allmol_m13_rdkit_sa_as = write_to_dict("m13_rdkit_sa_as_", suppl_m13_rdkit_sa_as)
allmol_m13_rdkit_sa_aa = write_to_dict("m13_rdkit_sa_aa_", suppl_m13_rdkit_sa_aa)
allmol_m13_rdkit_aa_aa = write_to_dict("m13_rdkit_aa_aa_", suppl_m13_rdkit_aa_aa)

### pre-screening

All the generated structures are pre-optimized with MM methods (UFF force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account non-hydrogen atoms only);
* compare the energies of conformers from pairs which are found to be similar (RMSD lower than the threshold);
* if these energies are too similar (the difference lower than the threshold), we will remove the conformer which has the higher energy value;

In [151]:
allmol_m13_rdkit = {}
allmol_m13_rdkit.update(allmol_m13_rdkit_sdf)
allmol_m13_rdkit.update(allmol_m13_rdkit_smi)
allmol_m13_rdkit.update(allmol_m13_rdkit_ss_sa)
allmol_m13_rdkit.update(allmol_m13_rdkit_ss_aa)
allmol_m13_rdkit.update(allmol_m13_rdkit_sa_sa)
allmol_m13_rdkit.update(allmol_m13_rdkit_sa_as)
allmol_m13_rdkit.update(allmol_m13_rdkit_sa_aa)
allmol_m13_rdkit.update(allmol_m13_rdkit_aa_aa)

print("The total number of generated conformers (before the duplicates removal) = ", len(allmol_m13_rdkit))
             
energy_m13_rdkit = {}
energy_m13_rdkit.update(e_m13_rdkit_sdf)
energy_m13_rdkit.update(e_m13_rdkit_smi)
energy_m13_rdkit.update(e_m13_rdkit_ss_sa)
energy_m13_rdkit.update(e_m13_rdkit_ss_aa)
energy_m13_rdkit.update(e_m13_rdkit_sa_sa)
energy_m13_rdkit.update(e_m13_rdkit_sa_as)
energy_m13_rdkit.update(e_m13_rdkit_sa_aa)
energy_m13_rdkit.update(e_m13_rdkit_aa_aa)

The total number of generated conformers (before the duplicates removal) =  453


In [152]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_rdkit = make_similarity_matrix(allmol_m13_rdkit)
similarity_matrix_rdkit_sorted = sorted(similarity_matrix_rdkit.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy
to_be_deleted = find_duplicates_in_sorted_similarity_matrix(similarity_matrix_rdkit_sorted, energy_m13_rdkit)

for mol in to_be_deleted:
    #print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_rdkit.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_rdkit[k]
    allmol_m13_rdkit.pop(mol, None)
    energy_m13_rdkit.pop(mol, None) 

In [153]:
print("We have removed potential conformer duplicates.")
print("The final number of remaining conformers = ", len(allmol_m13_rdkit))
print("Below we present all the remaining conformers aligned (to one aromatic ring).")

We have removed potential conformer duplicates.
The final number of remaining conformers =  53
Below we present all the remaining conformers aligned (to one aromatic ring).


In [154]:
align_structures_to_lowest_energy(allmol_m13_rdkit, energy_m13_rdkit)
p_rdkit = prepare_view(allmol_m13_rdkit)
p_rdkit.show()

In [155]:
print("Sorted energy of all selected conformers and the energy differences with respect to the lowest:")
energy_rdkit_sorted = sorted(energy_m13_rdkit.items(), key=lambda x: x[1])
energy_rdkit_diff = []
e_rdkit_min = energy_rdkit_sorted[0][1]
for e in energy_rdkit_sorted:
    e_diff = e[1] - e_rdkit_min
    energy_rdkit_diff.append([e[0], e[1], e_diff])

e_rdkit_df = pd.DataFrame(energy_rdkit_diff, columns=["conformer", "E", "E - Emin"])
e_rdkit_df

Sorted energy of all selected conformers and the energy differences with respect to the lowest:


,conformer,E,E - Emin
0,m13_rdkit_sa_aa_44,50.95,0.00
1,m13_rdkit_smi_51,53.09,2.14
2,m13_rdkit_smi_20,53.14,2.19
3,m13_rdkit_smi_10,53.22,2.27
4,m13_rdkit_smi_32,53.63,2.68
5,m13_rdkit_ss_aa_41,53.73,2.78
6,m13_rdkit_ss_aa_3,53.85,2.90
7,m13_rdkit_smi_12,53.87,2.92
8,m13_rdkit_smi_11,53.87,2.92
9,m13_rdkit_smi_31,54.06,3.11


## Summary

Now let's generate a list of all conformers (from all programs used) and align them. 
These conformers will further be used as starting points in DFT geometry optimizations

In [156]:
allmol_m13 = {}
allmol_m13.update(allmol_m13_b)
allmol_m13.update(allmol_m13_rdkit)

energy_m13 = {}
energy_m13.update(energy_m13_b)
energy_m13.update(energy_m13_rdkit)

The total number of conformers is:

In [157]:
print(len(allmol_m13))

107


In [158]:
align_structures_to_lowest_energy(allmol_m13, energy_m13)
p_all = prepare_view(allmol_m13)
p_all.show()

In [159]:
#Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". 
# It will be used to generate Gaussian inputs:

with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m13.items():
        f.write(key+"\n")

energy_sorted = sorted(energy_m13.items(), key=lambda x: x[1])
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13/detailed_list_selected_conformers_from_ballon_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1]))        